In [ ]:
import cv2
import numpy as np
import time

model_path = "E:\Object Detection\yolov3.weights"
config_path = "E:\Object Detection\yolov3.cfg"
# Load YOLO model
net = cv2.dnn.readNet(model_path, config_path)
layer_names = net.getLayerNames()
#output_layers = [layer_names[int(i[0]) - 1] for i in net.getUnconnectedOutLayersNames()]
output_layers = []
for i in net.getUnconnectedOutLayersNames():
    if not isinstance(i[0], int):
        print(f"Warning: Invalid layer name: {i}")
    else:
        output_layers.append(layer_names[int(i[0]) - 1])

# Initialize camera
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 720)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Function to calculate distance
def calculate_distance(box, focal_length, real_height, image_height):
    perceived_height = box[3]
    distance = (real_height * focal_length) / perceived_height
    return distance

# Parameters
focal_length = 1.0  # Length of the drone
real_height = 0.5  # width of the drone
prev_time = time.time()
prev_distance = None

while True:
    ret, frame = cap.read()
    height, width, channels = frame.shape

    # Prepare the image for YOLO
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Parse YOLO output
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:  # Confidence threshold
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Apply Non-max suppression to eliminate redundant overlapping boxes
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            color = (0, 255, 0)
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, label + " " + str(round(confidence, 2)), (x, y + 30), cv2.FONT_HERSHEY_PLAIN, 2, color, 2)
            
            # Calculate distance
            distance = calculate_distance([x, y, w, h], focal_length, real_height, height)
            cv2.putText(frame, f'Distance: {round(distance, 2)} m', (x, y + 50), cv2.FONT_HERSHEY_PLAIN, 2, color, 2)
            
            # Calculate speed
            current_time = time.time()
            if prev_distance is not None:
                speed = (prev_distance - distance) / (current_time - prev_time) * 3.6  # convert to km/h
                cv2.putText(frame, f'Speed: {round(speed, 2)} km/h', (x, y + 70), cv2.FONT_HERSHEY_PLAIN, 2, color, 2)
            prev_distance = distance
            prev_time = current_time

    cv2.imshow("Image", frame)
    key = cv2.waitKey(1)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()